# 📝 GYG Reviews



## 🧰 Librerías e importaciones

In [ ]:
import time
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

## 🛠️💻 Configuración del navegador

In [ ]:
options = Options()
options.add_argument("--start-maximized")
driver =
driver = webdriver.Chrome(service=
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

## 🧩 Funciones auxiliares

In [ ]:
def safe_text(el, selector):
    elems = el.find_elements(By.CSS_SELECTOR, selector)
    return elems[0].text.strip() if elems else ""

def cargar_todas_reseñas(max_scroll=100):
    """Clic en 'Ver más' y scroll hasta que no se carguen más reseñas"""
    tarjetas = []
    scroll_intentos = 0
    tarjetas_anteriores = 0

    while scroll_intentos < max_scroll:
        # clic en "Ver más" si aparece
        try:
            boton = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-test-id='activity-review-see-more-reviews-button']"))
            )
            driver.execute_script("arguments[0].scrollIntoView(true);", boton)
            time.sleep(2)  # espera antes de clicar
            driver.execute_script("arguments[0].click();", boton)
            print("🟢 Clic en 'Ver más reseñas'")
            time.sleep(4)  # espera extra después del clic
        except TimeoutException:
            pass  # si no hay botón, seguimos al scroll

        # scroll lento
        driver.execute_script("window.scrollBy(0, 800);")
        time.sleep(3)

        # actualizar tarjetas
        tarjetas = driver.find_elements(By.CSS_SELECTOR, "section.review-card[data-test-id='activity-review-card']")
        print(f"🔍 Tarjetas encontradas hasta ahora: {len(tarjetas)}")

        # detener si no hay tarjetas nuevas
        if len(tarjetas) == tarjetas_anteriores:
            print("⚠️ No se cargaron nuevas reseñas, deteniendo scroll")
            break

        tarjetas_anteriores = len(tarjetas)
        scroll_intentos += 1

    return tarjetas

## ➡️ Archivo de salida

In [ ]:
OUTPUT_FILE = "experiencias_reviews_getyourguide_mallorca.csv"

# Si ya existe, lo seguimos completando
file_exists = os.path.exists(OUTPUT_FILE)

## 🧩 CSV CON URLS

In [ ]:
df = pd.read_csv("experiencias_getyourguide_Mallorca.csv")  # columna 'enlace'
urls = df["enlace"].dropna().tolist()

## 🔎 Loop Principal

In [ ]:
for idx, url in enumerate(urls, 1):
    print(f"\n🔗 Procesando {idx}/{len(urls)}: {url}")
    driver.get(url)
    time.sleep(3)  # espera inicial lenta

    # Título actualizado según tu HTML
    try:
        titulo = driver.find_element(By.CSS_SELECTOR, "h1#adp-title-text").text.strip()
    except:
        titulo = ""

    tarjetas = cargar_todas_reseñas()  # función que descarga todas las reseñas
    print(f"🔍 Procesando {len(tarjetas)} reseñas de {titulo}")

    reseñas_temp = []

    for t in tarjetas:
        review_score = safe_text(t, "span.rating-star__label")
        autor_pais = safe_text(t, "span.review-card__author-details-name")
        if "–" in autor_pais:
            autor, pais = map(str.strip, autor_pais.split("–", 1))
        else:
            autor, pais = autor_pais, ""
        fecha = safe_text(t, "span.review-card___author-date")
        comentario = safe_text(t, "div.toggle-content__content")
        categoria = safe_text(t, "span.review-card__author-verified-label")

        if comentario:
            reseñas_temp.append([titulo, autor, pais, fecha, categoria, review_score, comentario, url])

    # Guardar al CSV inmediatamente
    if reseñas_temp:
        temp_df = pd.DataFrame(
            reseñas_temp,
            columns=["titulo", "autor", "pais", "fecha", "categoria", "review_score", "comentario", "enlace"]
        )
        temp_df.to_csv(
            OUTPUT_FILE,
            mode="a",
            index=False,
            header=not os.path.exists(OUTPUT_FILE) if not file_exists else False,
            encoding="utf-8-sig"
        )
        print(f"✅ {len(temp_df)} reseñas guardadas de {titulo}")

print("\n✅ Proceso terminado, todas las reseñas guardadas en CSV")
driver.quit()

## 🖱️⬇️↻ Preparación: scroll y paginación

In [ ]:
    tarjetas = cargar_todas_reseñas()  # función que descarga todas las
    tarjetas = cargar_todas_reseñas()  # función que descarga todas las reseñas
    print(f"🔍 Procesando {len(tarjetas)}
    print(f"🔍 Procesando {len(tarjetas)} reseñas de {titulo}")


    reseñas_temp = []

    for t in tarjetas:

        review_score = safe_text(t, "span.rating-star__label")
        autor_pais = safe_text(t, "span.
        autor_pais = safe_text(t, "span.review-card__author-details-name")
        if "–" in autor_pais:
            autor, pais = map(str.strip, autor_pais.split("–", 1))
        else:
            autor, pais = autor_pais, ""
        fecha = safe_text(t, "span.
        fecha = safe_text(t, "span.review-card___author-date")
        comentario = safe_text(t, "div.toggle-content__content")
        categoria = safe_text(t, "span.
        categoria = safe_text(t, "span.review-card__author-verified-label")

        if comentario:

            reseñas_temp.append([titulo, autor, pais, fecha, categoria,
            reseñas_temp.append([titulo, autor, pais, fecha, categoria, review_score, comentario, url])

    # Guardar al CSV inmediatamente
    if
    if reseñas_temp:
        temp_df =

## 🧼📊 Limpieza / transformación

In [ ]:
        temp_df = pd.DataFrame(


## 📝🔎 Scraping de reviews

In [ ]:
            reseñas_temp,
            columns=["titulo", "autor", "pais", "fecha", "categoria", "
            columns=["titulo", "autor", "pais", "fecha", "categoria", "review_score", "comentario", "enlace"]
        )
        temp_

## 🧼📊 Limpieza / transformación

In [ ]:
        temp_df.

## 💾📄 Guardado de resultados

In [ ]:
        temp_df.to_csv(
            OUTPUT_FILE,
            mode="a",
            index=False,
            header=not os.path.exists(OUTPUT_FILE) if not file_exists else False,
            encoding="utf-8-sig"
        )
        print(f"✅ {len(temp_df)}

## 📝🔎 Scraping de reviews

In [ ]:
        print(f"✅ {len(temp_df)} reseñas guardadas de {titulo}")

print("\n✅ Proceso terminado, todas las
print("\n✅ Proceso terminado, todas las reseñas guardadas en CSV")

## ✖️💻 Cerrar navegador

In [ ]:
driver.quit()